In [2]:
# packages
import os
import pandas as pd
import kaggle
from google.cloud import bigquery
import db_dtypes

### Step 1: Kaggle
##### Setup
1. Create a free Kaggle account
2. Go to Settings
3. Scroll down to the API section and select "Create New Token". This will download a kaggle.json file containing your API credentials
   - For Windows place your file here: C:\Users\<Your Username>\.kaggle\kaggle.json

##### Example Dataset
1. We will use the popular [Iris Species](https://www.kaggle.com/datasets/uciml/iris) dataset
   - Full url: https://www.kaggle.com/datasets/uciml/iris
2. We will need two parts
   - user who uploaded: uciml
   - name of dataset: iris

In [3]:
# dataset identifier
identifier = "uciml/iris"

# automatically looks for kaggle json file in user directory
kaggle.api.authenticate()

# path to current directory and unzip is true in case of multiple files
kaggle.api.dataset_download_files(identifier, path='.', unzip=True)

# get metadata (optional)
kaggle.api.dataset_metadata(identifier, path='.')

Dataset URL: https://www.kaggle.com/datasets/uciml/iris


'.\\dataset-metadata.json'

Within our current directory we will now see all iris files on Kaggle along with a metadata json file. In this case, a csv and sqlite file. Most datasets from kaggle do not include a database file (sqlite), only a flat file (csv), which is why I include Step 2.

In [4]:
# read csv
iris_df = pd.read_csv('iris.csv')
iris_df.head()
# iris_df.dtypes will need to check these for BQ write

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Step 2: Google BigQuery
##### Setup
1. Create a free GCP account
2. [Create a BigQuery Service Account Key File](https://docs.aws.amazon.com/dms/latest/sbs/bigquery-redshift-migration-step-1.html#:~:text=On%20the%20Service%20accounts%20page,for%20downloads%20in%20your%20browser)
   - place this json file in your current directory

In [5]:
# used for both BQ read/write

# Setting environmental variable directly in your code
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'bq-crudek-data.json'

# Initialize the BigQuery Client
client = bigquery.Client()

##### Write
- We will first write ```iris_df``` to our gcp project

In [6]:
# set table_id to the ID of the table to create
# table_id = "your-project.your_dataset.your_table_name"
table_id = 'crudek-data.practice_data.iris'

job_config = bigquery.LoadJobConfig(
    schema=[
        # "Species" column uses pandas dtype "object", so its data type is ambiguous and cannot be auto-detected.
        bigquery.SchemaField("Species", bigquery.enums.SqlTypeNames.STRING),
    ],
    # by default it appends the data. We can optionally change this with write_disposition 
    #write_disposition="WRITE_TRUNCATE",
)

# Make API request
job = client.load_table_from_dataframe(
    iris_df, table_id, job_config=job_config
)  
# Wait for the job to complete.
job.result()  

LoadJob<project=crudek-data, location=US, id=25562c58-6014-475f-b132-4fc739f4d171>

In [7]:
# confirm with shape
table = client.get_table(table_id)
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 300 rows and 6 columns to crudek-data.practice_data.iris


##### Read
- Now that ```iris_df``` is loaded in to BigQuery, we can query it with SQL

In [8]:
# write the SQL query
sql_query = """
    SELECT * 
    FROM `crudek-data.practice_data.iris` 
    LIMIT 10
"""

# Make an API request. 
query_job = client.query(
    sql_query)

# retreive and convert the result to a Pandas DataFrame
iris_df_2 = query_job.to_dataframe()

# Display the DataFrame
iris_df_2.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,99,5.1,2.5,3.0,1.1,Iris-versicolor
1,61,5.0,2.0,3.5,1.0,Iris-versicolor
2,80,5.7,2.6,3.5,1.0,Iris-versicolor
3,63,6.0,2.2,4.0,1.0,Iris-versicolor
4,93,5.8,2.6,4.0,1.2,Iris-versicolor
